In [ ]:
# !pip install ipynb
from ipynb.fs.full.useful_functions import *

In [ ]:
def get_circuit_parameters(subsets, verbose=False):
    
    # Build the graph.
    list_of_intersections = build_instance_graph(subsets, verbose=False)
    
    """
    The maximum number of controls needed in the circuit 
    is equal to the maximum number of intersections, i.e. the
    maximum number of edges coming out of a vertex.
    """
    num_max_ctrl = max([len(l) for l in list_of_intersections])
    
    NUM_ANC = num_max_ctrl - 1 # number of ancillas needed to use MCMTVChain
    QC_DIM = len(subsets) + NUM_ANC # total number of qubits needed

    if verbose:
        print("num_max_ctrl", num_max_ctrl)
        print("NUM_ANC", NUM_ANC)
        print("QC_DIM", QC_DIM)

    return list_of_intersections, num_max_ctrl, NUM_ANC, QC_DIM

## Build the cost operator (circuit)
### Paper di Wang et al.:
The objective is to find the $minimum$ of:
$$ H_P = -\lambda_1\sum_i^n w_i\frac{1-Z_i}{2} +\lambda_2 \sum_i^n\frac{1-Z_i}{2} = $$
$$  =- \lambda_1\sum_i^n \frac{w_i}{2} +\lambda_1\sum_i^n \frac{w_iZ_i}{2} +\lambda_2 \sum_i^n\frac{1}{2} -\lambda_2 \sum_i^n\frac{Z_i}{2} = $$
$$  = -A + \lambda_1\sum_i^n \frac{w_iZ_i}{2} + B - \lambda_2 \sum_i^n\frac{Z_i}{2} = $$
$$  = -A +B +\sum_i^n \frac{\lambda_1w_i -\lambda_2}{2}Z_i  $$
where
$$ A =  \frac{\lambda_1}{2}\sum_i^n w_i      $$ 
$$     B = \frac{ n \lambda_2}{2}  $$

In [ ]:
def build_cost_circuit(n, instance, k, verbose=False):
    
    U, subsets_dict = define_instance(n, instance, verbose=verbose)
    subsets = list(subsets_dict.values())
    
    _, _, _, QC_DIM = get_circuit_parameters(subsets, verbose=verbose)
    
    ###############################################################
    ###############################################################
    
    # z_op_on_first_qubit = 'Z'+'I'*(n-1)
    # labels = ['I'*NUM_ANC + ''.join(p) for p in distinct_permutations(z_op_on_first_qubit)]
    # print(labels)
    
    
    # Set the parameters.
    l2 = 1/(n * len(U) -2)
    l1 = k * n * l2 # l1/l2 must be equal to k*n

    A = l1 * sum([len(S) for S in subsets]) /2
    B = l2 * n /2
    
    
    # Create Z operators.
    coeffs = [(l1*len(S)/2 - l2/2) for S in subsets]
    Z_operators = [("Z", [i], coeffs[i]) for i in range(n)]
    hamiltonian = SparsePauliOp.from_sparse_list(Z_operators, num_qubits=QC_DIM)
    
    """
    Aggiungere la parte costante (-A+B) non serve a niente,
    dato che farò una minimizzazione. 
    """
    # # Add the -A+B term.
    # constant_hamiltonian = SparsePauliOp.from_list([("I"*QC_DIM, -A+B)])
    # hamiltonian = hamiltonian + constant_hamiltonian
    
    """Per tornare nel range di energia (0, -1], basterà
       calcolare E -A -B. 
                    E ---> E -A -B
    """
    if verbose:
        print("A =", A)
        print("B =", B)
        print("-A+B =", -A+B)
        print("\nhamiltonian:\n", hamiltonian)

    ###############################################################
    ###############################################################
    ### Creo un circuito per l'hamiltoniana 
    ### (in realtà funziona anche senza renderla un circuito)
    gamma = Parameter("gamma")
    evo = PauliEvolutionGate(hamiltonian, time=gamma)
     
    qc_ham = QuantumCircuit(QC_DIM)
    qc_ham.append(evo, range(QC_DIM))
    
    qc_ham = qc_ham.decompose(reps=2)
    qc_ham.draw(output="mpl", style="iqp")
    
    return A, B, hamiltonian, qc_ham

## Build the mixing operator (circuit)

In [ ]:
def build_mixing_circuit(n, instance, verbose=False):

    U, subsets_dict = define_instance(n, instance, verbose=verbose)
    subsets = list(subsets_dict.values())
    
    list_of_intersections, num_max_ctrl, NUM_ANC, QC_DIM = get_circuit_parameters(subsets, verbose=verbose)

    #######################################################################
    #######################################################################

    # Initialize the circuit.
    qr = QuantumRegister(n, 'q')
    anc = QuantumRegister(NUM_ANC, 'ancilla')
    qc_mixing = QuantumCircuit(qr, anc)
    
    """
    Inizializzare le ancille a 1 a ogni strato del QAOA non serve,
    basta inizializzarle una volta sola a p=1 perché su ogni ancilla agisce un 
    numero pari di NOT-gate in ogni strato.
    In realtà uno potrebbe inizializzarle in ogni strato per 
    proteggersi da eventuali bit-flip.
    """
    # # Initialize ancillas to 1.
    # for ancilla in range(n, QC_DIM):
    #     qc_mixing.initialize(1, ancilla)
    
    """
    Creo una lista di gate che (tramite VChain) implementano 
    X-rotazioni con un diverso numero di controlli. L'elemento i
    della lista avrà i+1 controlli.
    """
    beta = Parameter('beta')
    g = [MCMTVChain(RXGate(beta), x, 1) for x in range(1, num_max_ctrl+1)]
    gates = [g[i].to_gate() for i in range(len(g))]
    
    """
    Aggiungo al circuito i gate, specificando quali qubit
    devono fare da controlli: ricorda che l'ordine giusto è 
    [controlli, target, ancille] quindi se con 5 qubit [0,1,2,3,4] 
    e 2 ancille [5,6] voglio fare una rotazione X su 1 
    controllata da 0, 2, 3 scriverò:
    
    qc_mixing.append(gates[2], [0,2,3, 1, 5,6])
    """
    for i, intersections in enumerate(list_of_intersections):
        N = len(intersections)
        qubits_list = intersections + [i] + list(range(n, n+N-1))       
        qc_mixing.append(gates[N-1], qubits_list)
    
    qc_mixing.decompose().draw('mpl')
    
    return qc_mixing

### Build initialization circuit

In [ ]:
def build_initialization_circuit(n, instance, init_name, verbose=False):
    """
    Initialize with all0 or all1.
    
    Parameters
    ----------
        init_name (str): name of the initialization, e.g. "all1" or "all0"
    Return
    ------
        qc_initial (QuantumCircuit): circuit with the correct initialization
    """
    _, subsets_dict = define_instance(n, instance, verbose=verbose)
    subsets = list(subsets_dict.values())
    _, _, NUM_ANC, QC_DIM = get_circuit_parameters(subsets, verbose=verbose)

    
    ##### CIRCUIT #####
    qr = QuantumRegister(n, 'q')
    anc = QuantumRegister(NUM_ANC, 'ancilla')
    qc_initial = QuantumCircuit(qr, anc)
    
    
    ##### ANCILLAS #####
    #Initialize ancillas to 1.
    for ancilla in range(n, QC_DIM):
        qc_initial.initialize(1, ancilla)
    
    
    ##### QUBITS #####
    
    #### SCELGO GLI STATI CON CUI CREARE LA SOVRAPPOSIZIONE.
    #### RICORDA CHE DEVONO SODDISFARE IL VINCOLO, OVVERO DEVO
    #### SELEZIONARE INSIEMI CHE NON SI INTERSECANO
    # string = '0'*(n-2) + '11'
    # two_one_states = set(["".join(elem) for elem in distinct_permutations(string)])
    # print(two_one_states)
    # init_state = two_one_states + one_one_states
    # init_state = [EC]  + one_one_states

    if init_name == "all1":
        string = '0'*(n-1) + '1'
        one_one_states = ["".join(elem) for elem in distinct_permutations(string)]
        init_state =  one_one_states
    
    if init_name == "all0": 
        init_state = ["0"*n]
    
    
    # print("init_state:\n", init_state, "\ninit_name:", init_name)
    init_state = [x[::-1] for x in init_state] # reverse their order
    # occurrences = [1454, 1353, 37, 36, 34] # found on Leap
    
    
    #### SCELGO I PESI PER LA SOVRAPPOSIZIONE E GLI ASSEGNO AGLI STATI, 
    #### USANDO UN VETTORE DI LUNGHEZZA 2**n.
    # init_state = np.sqrt(1/len(init_state)) * np.ones(len(init_state)) # equal superposition
    # print(init_state)
    
    vec = np.zeros(2**n) # has length 2**n
    j = 0
    for i,nuple in enumerate(bit_gen(n)):
        state = "".join([str(bit) for bit in nuple]) # strings
        if state in init_state:
            vec[i] = 1
            j = j+1
    
    
    #### TRASFORMO IN STATEVECTOR IL VETTORE SCELTO ####
    state = Statevector(vec)
    qc_initial.initialize(state.data, list(range(n)), normalize=True) # set it as initial state for the first n qubits
    
    
    ##### MISURO PER CONTROLLARE CHE LA SOVRAPPOSIZIONE SIA CORRETTA #####
    ##### commenta per usare la sovrapposizione nel seguito! #####
    # qc_initial.measure_all()
    
    # # Let's see the result
    # svsim = Aer.get_backend('aer_simulator')
    # qc_initial.save_statevector()
    # result = svsim.run(qc_initial).result()
    
    # # Print the statevector neatly:
    # final_state = result.get_statevector()
    # plot_histogram(result.get_counts())
    
    # # from qiskit.visualization import plot_state_qsphere
    # # array_to_latex(final_state, prefix="\\text{Statevector = }")
    # # plot_state_qsphere(state)

    return qc_initial

## Compute energies (by enumeration) and plot the spectrum

In [ ]:
def show_spectrum(n, instance, k, verbose=False):
    U, subsets_dict = define_instance(n, instance, verbose=verbose)
    
    states, energies, states_feasible, energies_feasible, EXACT_COVERS = find_spectrum(U, subsets_dict, n, k)
    MEC = [state for state in EXACT_COVERS if state.count("1") == min([x.count("1")  for x in EXACT_COVERS])]
    
    print("EXACT_COVERS:", EXACT_COVERS)
    print("MEC:", MEC)

    #############################################################################
    #############################################################################
    
    #### PLOT ALL STATES ENERGY
    dim = 20
    plt.figure(figsize=(30,5))
    plt.rcParams['font.size'] = dim
    plt.title("All states")
    plt.plot(states, energies, '.--b')
    plt.xticks(rotation='vertical', fontsize=7)
    plt.xlabel("States")
    plt.ylabel("Energy")
    # plt.ylim(-3,0)
    plt.grid()
    
    # Highlight with red the exact covers
    highlight_correct_ticks(plt.gca(), states, EXACT_COVERS)
    
    plt.show()
    
    
    #### PLOT THE FEASIBLE STATES ENERGIES.
    dim = 20
    plt.figure(figsize=(30,5))
    plt.rcParams['font.size'] = dim
    plt.title("Feasible states")
    plt.plot(states_feasible, energies_feasible, '.--k')
    plt.xticks(rotation='vertical', fontsize=dim)
    plt.xlabel("States")
    plt.ylabel("Energy")
    # plt.ylim(-3,0)
    plt.grid()
    
    # Highlight with red the exact covers
    highlight_correct_ticks(plt.gca(), states_feasible, EXACT_COVERS)
    
    plt.show()

### Find the correct gamma bound by looking for the minimum eigenvalue of the cost hamiltonian

In [ ]:
def find_gamma_bound(n, instance, k, verbose=False):
    from itertools import combinations, groupby 
    from matplotlib.pyplot import MultipleLocator
    import math 

    U, subsets_dict = define_instance(n, instance, verbose=verbose)
    subsets = list(subsets_dict.values())
    
    l2 = 1/(n * len(U) -2)

    
    ### PRIMA RIORDINO IN BASE ALLA LUNGHEZZA
    how_many_elements = lambda x: len(x)
    subsets_ord = sorted(subsets, key=how_many_elements)
    
    B_array = np.arange(0,n+1)
    t = [0] + [sum([len(s) for s in subsets_ord[:i+1]]) for i,s in enumerate(subsets_ord)]
    
    distances = sorted(map(abs, B_array - k*n*np.array(t)), key=lambda x: x)
    a = l2*distances[1]
    
    gamma_bound = math.ceil(np.pi/a)
    gamma_bound_pi_units = math.ceil(1/a)

    if verbose:
        print("autovalore di modulo minimo: a = ", a)
        print("gamma massimo = math.ceil(2*np.pi/a) = ", math.ceil(2*np.pi/a))
        print(f"gamma bounds -> [0, {2*gamma_bound}] oppure [{-gamma_bound}, {gamma_bound}]")
        print(f"gamma bounds unità di pi-> [{-gamma_bound_pi_units}pi, {gamma_bound_pi_units}pi]")
        print(f"gamma_bound = {gamma_bound}")

    # #####################################
    # ### PLOT
    # plt.rcParams.update({'font.size': 13})
    # fig = plt.figure(figsize=(5,5))
    # ax = fig.add_subplot(111)
    # lengths = np.array([len(s) for s in subsets])
    # ax.plot(B_array, k*np.array(t), 'o-k', label='$t_{min}$ istanza 5')
    # ax.plot(B_array, np.arange(0,n+1), 'o-r', label='|B_array|')
    # ax.legend()
    # ax.grid()
    # ax.xaxis.set_major_locator(MultipleLocator(1))

    # ### POSSO FARE IL CHECK ANDANDO A CERCARE IL MINIMO DELLE ENERGIE CALCOLATE
    # new_states_energy = [(s,abs(e)) for (s,e) in zip(states_feasible, energies_feasible) if s!= '0'*n ]
    # new_energy = [tup[1] for tup in new_states_energy]
    # for s,e in new_states_energy:
    #     if e == min(new_energy):
    #         print("state =", s, " with |energy| = a =", e)
    #         print("gamma_max = 2*np.pi/|energy| =", 2*np.pi/e)
    #         print("gamma_max in unità di pi greco = 2/|energy| =", 2/e)
    #         g = math.ceil(2*np.pi/e)
    #         print(f"gamma bounds-> [{-math.ceil(np.pi/e)}, {math.ceil(np.pi/e)}]")
    #         print(f"gamma bounds-> [{-math.ceil(1/e)}pi, {math.ceil(1/e)}pi]")
    #         break
    
    return gamma_bound